# Explainer Notebook - E-Sports (League of Legends)

# Code for data analysis (Rest of the explainer notebook is below)
Here we just import our libraries and read our dataset

In [1]:
import numpy as np, pandas as pd, matplotlib as plt
from matplotlib import pyplot as py
import inspect, os, math

match_data = pd.read_csv("F:\Download\matchinfo.csv")

This section defines all the possible leagues, and filters some of them out, as we are only interested in the major regions and the two international tournaments (MSI and WC). We also define our year range to go from 2015 to 2017, as the data for 2018 is incomplete and thus not usable.

In [2]:
seasons = match_data.Season.unique()
years = range(2015,2018)

We define a new data set where we group the individual teams into their respective reagions to make it easier to compare performances between regions on an international level. For this data set we also only consider MSI and WC.

In [3]:
league_data = match_data.loc[(match_data['League'] == 'WC') | (match_data['League'] == 'MSI')]
lckteam = match_data.loc[(match_data['League'] == 'LCK')].blueTeamTag.unique()
lckteam = np.append(lckteam,['SSW']) #Specialcase
nateam = match_data.loc[(match_data['League'] == 'NALCS')].blueTeamTag.unique()
euteam = match_data.loc[(match_data['League'] == 'EULCS')].blueTeamTag.unique()
lmsteam = match_data.loc[(match_data['League'] == 'LMS')].blueTeamTag.unique()
cblolteam = match_data.loc[(match_data['League'] == 'CBLoL')].blueTeamTag.unique()

Here we define alle the teams that are either part of the chinese league (for which the data is mostly absent) or were special teams for a single tournament match (ALL = Allstars), or simply placeholders. We then put then in an LPL placeholder which gets ignored later.

In [4]:
others = ['EDG', 'DP', 'SHR', 'LMQ', 'OMG', 'ALL', 'IG', 'NWS', 'BKT', 'LGD', 'ANX', 'RNG', 'GAM', 'WE', 'BJK', 'SUP', 'Winners'] # Was necessary before but can be deleted now (Just use the difference between all data and the other regions) 

Here all teams get put into their respective regions to calculate the individual region's performance at international tournaments.

In [5]:
league_data.loc[league_data['blueTeamTag'].isin(lckteam), 'blueTeamTag'] = 'LCK'
league_data.loc[league_data['blueTeamTag'].isin(nateam), 'blueTeamTag'] = 'NALCS'
league_data.loc[league_data['blueTeamTag'].isin(euteam), 'blueTeamTag'] = 'EULCS'
league_data.loc[league_data['blueTeamTag'].isin(lmsteam), 'blueTeamTag'] = 'LMS'
league_data.loc[league_data['blueTeamTag'].isin(cblolteam), 'blueTeamTag'] = 'CBLoL'
league_data.loc[league_data['blueTeamTag'].isin(others), 'blueTeamTag'] = 'LPL'
league_data.loc[league_data['redTeamTag'].isin(lckteam), 'redTeamTag'] = 'LCK'
league_data.loc[league_data['redTeamTag'].isin(nateam), 'redTeamTag'] = 'NALCS'
league_data.loc[league_data['redTeamTag'].isin(euteam), 'redTeamTag'] = 'EULCS'
league_data.loc[league_data['redTeamTag'].isin(lmsteam), 'redTeamTag'] = 'LMS'
league_data.loc[league_data['redTeamTag'].isin(cblolteam), 'redTeamTag'] = 'CBLoL'
league_data.loc[league_data['redTeamTag'].isin(others), 'redTeamTag'] = 'LPL'
leagues = league_data.blueTeamTag.unique()

E:\Anaconda\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


## Get team performance within own region
Here we calculate the performance between all teams in a region. We only account for all the teams that were active during all the seasons that we have full data for (6 seasons, 2 for each year).

In [6]:
columns = ['League', 'Date', 'Team', 'GamesPlayed', 'GamesWon', 'WinRatio']
total_matchup_data = pd.DataFrame(columns=columns)
for league in leagues:    
    teams_r = match_data.loc[(match_data['League'] == league)].blueTeamTag.unique()
    teams_b = match_data.loc[(match_data['League'] == league)].redTeamTag.unique()
    teams = set(np.concatenate((teams_r,teams_b)))
    for year in years:
        for season in seasons:
            for team in teams:
                nGames_mask = ((match_data['Year'] == year) & (match_data['League'] == league) & (match_data['Season'] == season) & ((match_data['redTeamTag'] == team) | (match_data['blueTeamTag'] == team)))
                win_mask = (((match_data['blueTeamTag'] == team) & (match_data['bResult'] == 1)) | ((match_data['redTeamTag'] == team) & (match_data['rResult'] == 1)))
                games_played = match_data.loc[nGames_mask]
                games_won = games_played.loc[win_mask]
                if(len(games_played.index) <= 0):
                    continue;
                win_ratio = (len(games_won.index)/(len(games_played.index)))
                date = str(year) + "-" + ("01" if season == "Spring" else "06")
                row = pd.DataFrame([[league,date,team,len(games_played.index), len(games_won.index), win_ratio]], columns=columns)
                total_matchup_data = total_matchup_data.append(row)
total_matchup_data = total_matchup_data.groupby("Team").filter(lambda x: len(x) >= 6)

# Compute International performance for each team for each split
Here we compute how each region how they perform against other regions in international tournaments, that is the World Championship (WC) and the Mid-Season-Invitational (MSI).

In [7]:
Tournaments = ['WC', 'MSI']

In [8]:
int_columns = ['League', 'Year', 'Tournament', 'GamesPlayed', 'GamesWon', 'WinRatio']
international_matchup_data = pd.DataFrame(columns=int_columns)
for t in Tournaments:
    for league in leagues:
        for year in years:
            nGames_mask = ((league_data['Year'] == year) & (league_data['League'] == t) & ((league_data['redTeamTag'] == league) | (league_data['blueTeamTag'] == league)))
            win_mask = (((league_data['blueTeamTag'] == league) & (league_data['bResult'] == 1)) | ((league_data['redTeamTag'] == league) & (league_data['rResult'] == 1)))
            games_played = league_data.loc[nGames_mask]
            if(len(games_played.index) <= 0):
                continue;
            games_won = games_played.loc[win_mask]
            win_ratio = (len(games_won.index)/(len(games_played.index)))
            row = pd.DataFrame([[league,year,t,len(games_played.index), len(games_won.index), win_ratio]], columns=int_columns)
            international_matchup_data = international_matchup_data.append(row)

## Compute team matchup data

Here we compute all possible matchups between all teams in the North American League (NALCS) and calculate the Win/Loss ratio. We only consider combinations of teams that actually have atleast played once against eachother. To find all teams we must first go through the blue and red teams (Each match has one team as the blue team and one as the red team).

In [9]:
team_col = ['Team1','Team2', 'GamesPlayed', 'Team1GamesWon', 'Team1WinRatio']
team_match_data = pd.DataFrame(columns=team_col)
teams_r = match_data.blueTeamTag.unique()
teams_b = match_data.redTeamTag.unique()
# Find all possible teams from among the red team and blue teams.
teams = set(np.concatenate((teams_r,teams_b)))
teams = {x for x in teams if pd.notna(x)}
matchups = []
# Find all possible matchup combinations
for team1 in teams:
    for team2 in teams:
        if(team1 != team2):
            matchups.append((team1,team2))  
# Find all unqiue matchups
matchups = set(tuple(sorted((a,b))) for a,b in matchups)         
for team1, team2 in matchups:
            nGames_mask = ( (match_data['League'] =='NALCS') & ((match_data['redTeamTag'] == team1) | (match_data['blueTeamTag'] == team1)) & ((match_data['redTeamTag'] == team2) | (match_data['blueTeamTag'] == team2)))
            win_mask = (((match_data['blueTeamTag'] == team1) & (match_data['bResult'] == 1)) | ((match_data['redTeamTag'] == team1) & (match_data['rResult'] == 1)))
            games_played = match_data.loc[nGames_mask]
            if(len(games_played.index) <= 0):
                continue;
            games_won = games_played.loc[win_mask]
            win_ratio = (len(games_won.index)/(len(games_played.index)))
            row = pd.DataFrame([[team1,team2,len(games_played.index), len(games_won.index), win_ratio]], columns=team_col)
            team_match_data = team_match_data.append(row)
team_match_data = team_match_data.groupby("Team1").filter(lambda x: len(x) >= 6)

## Write data to files

In [10]:
total_matchup_data.to_csv("TotalMatchupData.csv",index=False)
international_matchup_data.to_csv("InternationalMatchupData.csv",index=False)
team_match_data.to_csv("FocusedTeamMatchData.csv",index=False)

# Explanations

## 1. Motivation

### 1.1 What is your dataset?

This dataset contains information about most of the professional E-Sports matches for the game League of Legends from the years 2015 to 2017 (the data for 2018 is incomplete). Each row in the dataset is a single professional match. There are different leagues and the league a team belongs to is decided by their location. For example: North American teams would play in the North American League Championsship Series (NALCS). Each year is divided into 2 seasons: Spring and Summer. At the end of each season there is an international tournament (WC or MSI) were teams from all regions compete. The information for the chinese league (LPL) is incomplete and thus removed. Matches from all these seasons and tournaments are represented in our dataset.
Note: For more information about what E-Sports and League of Legends are, please refer to the introduction in the Observable Notebook.


### 1.2 Why did you choose this/these particular dataset(s)?

Unlike traditional sports like football or baseball, there has not yet been a lot of data analysis for E-Sports, even for very popular game like League of Legends. On top of this, there is also not much data readily available. It is possible to get live data form current games but older data (even just a few years back) is harder to come by. This isn't too surprising as  it is a relatively new field. This is also one reasons we decided to look at this type of data in particular as we thought it would be interesting to explore this new field. This particular dataset was retreived from https://www.kaggle.com/chuckephron/leagueoflegends. 

### 1.3 What was your goal for the end user's experience?

We wanted to expose the user to an area that they possibly don't know much (or anything) about. We want to show and explain the user what E-Sports is and show them a glimpse into how professional performance has changed over the years. As we cannot assume that the user knows anything about the game, we have to be careful to not get too technical about specific details from the game and keep it more approachable.

## 2. Basic Stats

### 2.1 Write about your choices in data cleaning and preprocessing

Before any actual work on the data is performed it needed to be validated. While getting all the actual data for the games is difficult, checking if some piece of data about the game is valid is much easier. The developer of the game (Riot Games) has released a very simple API that allows developers to extract some information about the game. One column in the data (Address) contains a link to the offical website and a special id for that particular match. Using that ID and the API we could retreive some information about the match directly from the devleoper of the game to check if the data in the set was actually valid, which it was.

Apart from this, we also needed to remove several teams from the set. There were several reasons for this. The first was that some of the teams were from the chinese league, for which the data was incomplete, so we decided to focus on the regions for which the data was actually complete.  There were also placeholder names for teams for some matches (Which were categorized as "Winners" and "ALL"). The last teams to be removed were teams that had a comparatively low amount of games throughout the whole set. The reason for that is that sometimes teams give up their spot in the League, either because they performed poorly or could not be financially sustained.

For some of the visualizations we only look at teams that have been performing throughout all three years. The reasoning for this is that when we want to look at how a team's performance has changed over the years it would not make much sense to look at teams that have only competed in one or two seasons. Using the same timeframe for all teams makes sense when we want to compare them all to one another.

### 2.2 Write a short section that discusses the dataset stats, containing key points/plots from your exploratory data analysis

The following graphs show some of the basic stats. The first barchart shows the performance between the different regions, highlighting the difference in performance among the regions, which is also discussed in the dataviz. The next chart shows how performance of a specific team has changed over the seasons. We wanted to use this to hightlight how some teams struggle through some seasons, as the biggest changes in the game are made between seasons. In the dataviz itself we tried to highlight this change for all relevant teams, that is teams that have performed through all three sampled years.

## 3. Data Analysis

### 3.1 Describe your data analysis and explain what you've learned about the dataset

While analyzing the data we first wanted to identify outliers, that is teams that were only ocurring in very few matches compared to the rest, so we could remove them before the actual analysis. We then identified that there are big there is a definite difference in performance between the regions, with the korean league (LCK) at the very top. Looking the performance of all teams over the year we can see that many of them have very varying performance over that time span, while only very few performed more or less consistent. Another interesting thing learned was how big the difference between the top teams and the lower performing teams was. Whilesom top teams could have a Win/Loss ratio up to around 90%, there were some team that struggled to even win a single game.

## 4. Genre

### 4.1 Which genre of data story did you use?
The genres used for the dataviz are magazine and annotated charts. Here the annotated charts are the interactive diagrams and the magazine style are the text description of the topic. This combination give the user the possibility of interacting with the diagram themself in their own tempo, without letting the user find all the information themself. There is also a Alluvial diagram which is a type of flowchart, but are used as an annotated chart in the observable to let the user interact with it.

### 4.2 Which tools did you use from each of the 3 categories of Visual Narrative (Figure 7 in Segal and Heer). Why?
**Visual Structuring**: The viewer is provided with a **consistent visual platform** to clearly communicate our narritice and don't get the user too lost in the visuals.

**Highlighting**: For the multi-line diagram **Feature Distinction** are used, as the user have the option of placing their mouse over each of the line, to get it highlighted and show the name of the team.

**Transition Guidance**: None of the tools have been used for transition of the diagrams. 

### 4.3 Which tools did you use from each of the 3 categories of Narrative Structure (Figure 7 in Segal and Heer). Why?
**Ordering**: The narrativ are in a **linear** fashion to help guide the user through the data and giving extra explanations as needed. The reasoning again is the reader might not have much knowledge about the subject so it would we unwise to make the narration too open ended.

**Interactivity**: The user is provided with the possibility to get more details about the details by **Hover Highlighting / Details** and giving the possibility to **Filter** some of the charts based on the League they want to inspect to give them the possibility to get a closer look on the things they are interested in.

**Messaging**: The dataviz contains an **Introductory Text** to explain the subject to readers who do not know much about it, whereafter the story is complement the visualizations with an **Accompanying Article** that explains our findings and reasonings to the reader.

## 5. Visualizations

### 5.1 Explain the visualizations you've chosen and why are they right for the story you want to tell?.
For the first visualizations simple barcharts are used to show the performance between the regions (On the left) and between teams in different regions (On the right). The purpose for that is to start out with presenting a simple overview for performance between teams and regions and because there were not many different teams within a region, a barchart would be good fit, also because they made it easy to see if there were extreme outliers (like the LCK league in the international comparison).

The reasoning for the Multi-Line chart was that we wanted to achieve two things. We wanted to show the user the general tendencies for how and if the performance of the teams changed significantly throughout the seasons, so it makes sense to have them all presented together. However, we also wanted to give the user the possibility to explore this data and look at which teams are potential outlies by hovering over their respective line. Outliers in this case could for example be teams with a extremely stable performance throughout all years or teams this did extremely poorly in one season but excelled in another.

The last diagram is used to let the user get a look at how we can use the data to guess future performances. If they look at the flow of a particular team they can follow how they would perform throughout a season based on previous results. Of course these predictions have to be taken with a grain of salt. As stated the game constantly changes but as it is very hard/impossible to predict how the game is changed in the future, this is still one of the better ways to do it.

## 6. Discussion

### 6.1 What went well?
We believe that the dataviz presented does a good job with presenting this field that not all may familiar with, without it being too confusing. We also think that the visualizations chosen do a good job at presenting the data in a way that makes sense. 

### 6.2 What is still missing? What could be improved?, Why?
Because of the complexity of the game and the extra amount of data (Like minute-to-minute precise game data) for each match that could be extracted from the earlier mentioned API or other third-party sources, there could be added a lot more depth to the analysis. This additional data could then also be used for machine learning, which we could then use to predict the outgame of a game based on both teams' performance throughout the first half of the game based on the data that has been collected until then. We did willfully focus on more basic parts because we wanted the dataviz to be accessible for everyone but if the focus of the dataviz would be more on users already familiar with the field, then these things should certainly be considered. Another thing that could be improved is the visual factor. The dataviz itself could be a little bit more visually E-Sports themed and in general visually appealing to increase the user's interest in exploring it.


## 7. Contributions. Who did what?

In general, both members have contributed to all parts of the assignment but our main work was distributed as follows:

**Simon (s153694)**: Main responsible for the first 2 of the 3 visualizations (The bar charts and Multi-Line chart) as well as the introductory chapter in the observable notebook.


**Arne (s182839)** : Main responsible for the data analysis, the explainer notebook and the third visualization (Alluvial Diagram).